In [ ]:
!pip install openai-whisper ffmpeg torch

In [ ]:
from google.colab import drive

def mount_google_drive():
    """Google Driveをマウントします"""
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")

mount_google_drive()

In [ ]:
import subprocess

def extract_audio_from_video(video_file, output_audio_file):
    """MP4ファイルから音声を抽出します"""
    command = f"ffmpeg -i {video_file} -q:a 0 -map a {output_audio_file} -y"
    subprocess.run(command, shell=True)
    print(f"Audio extracted to {output_audio_file}")

# 例: 動画ファイルを音声ファイルに変換
video_file = "/content/drive/My Drive/TranscriptionFolder/sample_video.mp4"  # Google DriveのMP4ファイルパス
output_audio_file = video_file.replace(".mp4", ".mp3")
extract_audio_from_video(video_file, output_audio_file)

In [ ]:
import os

def split_audio_file(audio_file, output_dir, segment_length=300):
    """音声ファイルを指定された長さ（秒）で分割"""
    os.makedirs(output_dir, exist_ok=True)
    command = f"ffmpeg -i {audio_file} -f segment -segment_time {segment_length} -c copy {output_dir}/output%03d.mp3"
    subprocess.run(command, shell=True)
    print(f"Audio split into segments in {output_dir}")

# 例: 5分（300秒）ごとに分割
split_audio_file(output_audio_file, "/content/drive/My Drive/TranscriptionFolder/audio_segments", segment_length=300)

In [ ]:
import whisper

def list_audio_files(folder_path):
    """指定フォルダ内の音声ファイルをリストアップします"""
    return [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith('.mp3')
    ]

def transcribe_files(files, model):
    """ファイルごとにテキスト化を実行し、結果を保存します"""
    for file_path in files:
        print(f"Processing: {file_path}")
        result = model.transcribe(file_path)
        
        # テキスト結果を保存
        output_file = file_path.replace('.mp3', '.txt')
        with open(output_file, "w") as f:
            f.write(result["text"])
        print(f"Saved transcription to: {output_file}")

# 分割された音声ファイルをリストアップ
segment_folder = "/content/drive/My Drive/TranscriptionFolder/audio_segments"
audio_files = list_audio_files(segment_folder)

# Whisperモデルをロード
model = whisper.load_model("small")  # 必要に応じてモデルサイズを変更

# テキスト化を実行
if audio_files:
    transcribe_files(audio_files, model)

In [ ]:
print("すべてのファイルをテキスト化しました！")